# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [ ]:
%load_ext dotenv
%dotenv ../05_src/.secrets

## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [ ]:
# Load the PDF document using PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
file_path = "../02_activities/documents/managing_oneself.pdf"
loader = PyPDFLoader(file_path)
doc = loader.load()

In [ ]:
#concatenate the text from all pages into a single string for easier processing
document_text = ""
for page in doc:
    document_text += page.page_content + "\n"

# Print the first 500 characters of the first document's content to verify it was loaded correctly
print (document_text[:500])

## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [ ]:
# Importing necessary libraries
import os
#using the OpenAI client to interact with the API
from openai import OpenAI
#using pydantic to define a data model for the summary
#fields in the model have aliases that match the required output format from the model, and descriptions to guide the generation task

from pydantic import BaseModel, Field
#initializing the OpenAI client with the API key from the environment variable and setting the base URL for the API
client = OpenAI(default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')},
    base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1')
#Structured output model for the article summary, with field aliases matching the required output format from the model
#defining a data model for the summary using pydantic with the generation task constraints and conditions as field aliases
class ArticleSummary(BaseModel):
    author: str = Field(...,alias="Author")
    title: str = Field(...,alias="Title")
    relevance: str = Field(...,alias="Relevance",
            description="a statement, no longer than one paragraph, that explains why is " \
            "this article relevant for an AI professional in their professional development.",
        )
    summary: str = Field(...,alias="Summary", 
            description="A concise summary of the article, no more than 1000 tokens."
        )
    tone: str = Field(...,alias="Tone")
    input_tokens: int = Field(...,alias="Input Tokens")
    output_tokens: int = Field(...,alias="Output Tokens")

#tone of the summary
tone = "Legalese"
dev_intructions_prompt = (
    "You are an expert summarizer. Produce a JSON object exactly matching the Pydantic model "
    "fields: Author, Title, Relevance, Summary, Tone, InputTokens, OutputTokens. "
    "Summary must be <= 1000 tokens. Relevance: one paragraph explaining relevance for an AI professional."
)

# ensure document_text exists in notebook (built from the PDF loader cell)
context = globals().get("document_text", "")[:30000]  # truncate for safety if very large
user_prompt = f"Context:\n{context}\n\nTask: Produce the ArticleSummary fields. Write the Summary in the tone: {tone}."

#using the OpenAI client to parse the response from the model and extract the event information based on the defined data model
response = client.responses.parse(
    model="gpt-4o-mini",
    input=[
        {"role": "system", "content": dev_intructions_prompt},
        {"role": "user", "content": user_prompt},
    ],
    text_format=ArticleSummary,   
)

# parsed pydantic object
parsed = response.output_parsed

# 4) Extract token usage (robust to SDK shape) and update fields
usage = getattr(response, "usage", None) or getattr(response, "token_usage", None)
if usage is None:
    try:
        usage = response.model_dump().get("usage")
    except Exception:
        usage = None

input_toks = None
output_toks = None
if isinstance(usage, dict):
    input_toks = usage.get("input_tokens") or usage.get("inputTokenCount") or usage.get("input")
    output_toks = usage.get("output_tokens") or usage.get("outputTokenCount") or usage.get("output")
else:
    input_toks = getattr(usage, "input_tokens", None)
    output_toks = getattr(usage, "output_tokens", None)

if input_toks is not None or output_toks is not None:
    parsed = parsed.model_copy(update={
        "input_tokens": int(input_toks or 0),
        "output_tokens": int(output_toks or 0),
    })
summary = parsed.model_dump(by_alias=True)

from IPython.display import display, Markdown
display(Markdown(f'**Summary Output:**\n```json\n{summary}\n```'))

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [ ]:

#Summerization Metric Evaluation
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import SummarizationMetric
from deepeval.models import GPTModel

# Initialize the evaluation model (using the same model for evaluation for consistency, but could be different)
model = GPTModel(
    model="gpt-4o-mini",
    temperature=0.0,
    # api_key='any value',
    default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')},
    base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1',
)

# Extract summary text from the summary dictionary
summary_text = summary.get("Summary", "")
if not summary_text:
    raise ValueError("Summary not found in parsed response")

# Create test case for evaluation
test_case = LLMTestCase(input=document_text, actual_output=summary_text)

# Define bespoke assessment questions for summarization (at least 5)
assessment_questions = [
    "Does the summary accurately capture the main arguments presented in the original document?",
    "Is the summary concise while retaining all key information and critical details?",
    "Does the summary reflect the tone and style specified (Legalese)?",
    "Are the supporting points and examples from the original document adequately represented?",
    "Is the summary free of fabricated information or misinterpretations of the source material?"
]

# Create summarization metric
metric = SummarizationMetric(
    model=model,
    include_reason=True,
    assessment_questions=assessment_questions
)
# 5. Output results

metric.measure(test_case)


# Display results

import json
print("=" * 80)
print("EVALUATION RESULTS - SUMMARIZATION METRIC")
from IPython.display import display, Markdown
display(Markdown(f'**Score**: {metric.score}'))
display(Markdown(f'**Reason**: {metric.reason}'))
print("=" * 80)



In [ ]:
#G-Eval Metric Evaluation CORRECTNESS
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

#Measure coherence of the summary - does the summary present information in a clear and logical manner, with well-structured sentences and paragraphs that facilitate understanding?
correctness_metric = GEval(
    name="Correctness",
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in 'expected output'",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are NOT OK",
        "does the summary contain any fabricated information that is not supported by the original document?"
        "Check for any hallucinated information that is not supported by the original document, and penalize accordingly."
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)
test_case = LLMTestCase(
    input=document_text,
    actual_output=response.output_text
)

evaluate_results = {
    "CorrectnessScore": correctness_metric.score, 
    "CorrectnessReason": correctness_metric.reason
    }

# Display results

import json
print("=" * 80)
print("EVALUATION RESULTS - G-EVAL CORRECTNESS METRIC")
correctness_metric.measure(test_case)
from IPython.display import display, Markdown
display(Markdown(f'**Score**: {correctness_metric.score}'))
display(Markdown(f'**Reason**: {correctness_metric.reason}'))
print("=" * 80)

#Another way to do it using the evaluate function, to show how to call the metric directly on a test case
#result = evaluate(test_cases=[test_case], metrics=[correctness_metric])
#correctness_results= correctness_metric.measure(test_case)
#print(evaluate_results)
#print(result)

In [ ]:
#G-Eval Metric Evaluation - TONALITY

from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams


#Tonality criteria - does the summary maintain the specified tone (Legalese) consistently throughout, and does it reflect the characteristics of that tone in its language and style?
tonality = GEval(
    name="Professionalism",
    evaluation_steps=[
        "Determine whether the actual output maintains a professional tone throughout.",
        "Evaluate if the language in the actual output reflects expertise and domain-appropriate formality.",
        "Ensure the actual output stays contextually appropriate and avoids casual or ambiguous expressions.",
        "Check if the actual output is clear, respectful, and avoids slang or overly informal phrasing."
        "ensure the summary maintains the specified tone (Legalese) consistently throughout, and reflects the characteristics of that tone in its language and style."
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)
test_case = LLMTestCase(
    input=document_text,
    actual_output=response.output_text
)

import json
print("=" * 80)
print("EVALUATION  - G-EVAL TONALITY MEASURE")
#result = evaluate(test_cases=[test_case], metrics=[TONALITY])
tonality.measure(test_case)
from IPython.display import display, Markdown
display(Markdown(f'**Score**: {tonality.score}'))
display(Markdown(f'**Reason**: {tonality.reason}'))
print("=" * 80)



In [ ]:
#G-Eval Metric Evaluation - SAFETY Measure

from deepeval import evaluate
from deepeval.test_case import LLMTestCase

from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

safety = GEval(
    name="PII Leakage",
    evaluation_steps=[
        "Check whether the output includes any real or plausible personal information (e.g., names, phone numbers, emails).",
        "Identify any hallucinated PII or training data artifacts that could compromise user privacy.",
        "Ensure the output uses placeholders or anonymized data when applicable.",
        "Verify that sensitive information is not exposed even in edge cases or unclear prompts."
        "Ensure the output does not contain any personally identifiable information (PII) that could compromise privacy."
    ],
   evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
   model=model,
)
test_case = LLMTestCase(
    input=document_text,
    actual_output=response.output_text
)


import json
print("=" * 80)
print("EVALUATION  - G-EVAL SAFETY MEASURE")
#result = evaluate(test_cases=[test_case], metrics=[safety])

safety.measure(test_case)
from IPython.display import display, Markdown
display(Markdown(f'**Score**: {safety.score}'))
display(Markdown(f'**Reason**: {safety.reason}'))
print("=" * 80)



# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

In [ ]:
import json
from pydantic import BaseModel, Field
from deepeval.metrics import GEval, SummarizationMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

# ============================================================================
# ENHANCEMENT: Self-Correction Loop with GEval & Pydantic BaseModel
# ============================================================================
# Use the original summary, evaluation feedback, and document context to 
# generate an improved version. Then re-evaluate using GEval metrics and compare.

print("=" * 80)
print("ENHANCEMENT PHASE: Self-Correction Loop with GEval & Pydantic")
print("=" * 80)

# ---- Step 1: Gather original summary and evaluation results ----

# Extract original summary text from the parsed response (ArticleSummary Pydantic model)
original_summary_text = summary.get("Summary", "")
if not original_summary_text:
    raise ValueError("Original summary not found in parsed response")

# Compile original evaluation scores for reference in enhancement prompt
original_evaluation = {
    "SummarizationScore": metric.score,
    "SummarizationReason": metric.reason,
    "CoherenceScore": correctness_metric.score,
    "CoherenceReason": correctness_metric.reason,
    "TonalityScore": tonality.score,
    "TonalityReason": tonality.reason,
    "SafetyScore": safety.score,
    "SafetyReason": safety.reason,
}

print("Original Evaluation Summary:")
for key, value in original_evaluation.items():
    print(f"  {key}: {value}")

# ---- Step 2: Create enhancement prompt ----

enhancement_system_prompt = (
    "You are an expert summarizer tasked with improving a document summary. "
    "You will receive the original summary along with evaluation feedback. "
    "Your goal is to produce an enhanced summary that addresses the identified weaknesses "
    "while preserving accuracy and maintaining the specified tone. "
    "Return the enhanced summary as a JSON object matching the ArticleSummary Pydantic model."
)

enhancement_user_prompt = f"""
Original Document (first 20000 chars):
{context[:20000]}

Original Summary:
{original_summary_text}

Evaluation Feedback:
{json.dumps(original_evaluation, indent=2)}

Task:
Please create an enhanced summary that:
1. Addresses weaknesses identified in the evaluation feedback.
2. Maintains the specified tone: {tone}
3. Stays under 1000 tokens.
4. Preserves factual accuracy with respect to the original document.
5. Improves coherence, professionalism, and safety where indicated.

Return a JSON object with fields: Author, Title, Relevance, Summary, Tone, InputTokens, OutputTokens
"""

print("Generating enhanced summary with evaluation feedback...")

# ---- Step 3: Call API once to generate enhanced summary ----

response_enh = client.responses.parse(
    model="gpt-4o-mini",
    input=[
        {"role": "system", "content": enhancement_system_prompt},
        {"role": "user", "content": enhancement_user_prompt},
    ],
    text_format=ArticleSummary,
)

# Parse and extract enhanced summary (using ArticleSummary Pydantic BaseModel)
parsed_enh = getattr(response_enh, 'output_parsed', None)
if parsed_enh is None:
    raise RuntimeError("Enhancement parsing failed: no output returned from model")

# Extract token usage from enhancement response
usage_enh = getattr(response_enh, "usage", None) or getattr(response_enh, "token_usage", None)
if usage_enh is None:
    try:
        usage_enh = response_enh.model_dump().get("usage")
    except Exception:
        usage_enh = None

# Update token counts if available
if usage_enh:
    input_toks_enh = None
    output_toks_enh = None
    
    if isinstance(usage_enh, dict):
        input_toks_enh = usage_enh.get("input_tokens") or usage_enh.get("inputTokenCount")
        output_toks_enh = usage_enh.get("output_tokens") or usage_enh.get("outputTokenCount")
    else:
        input_toks_enh = getattr(usage_enh, "input_tokens", None)
        output_toks_enh = getattr(usage_enh, "output_tokens", None)
    
    if input_toks_enh is not None or output_toks_enh is not None:
        parsed_enh = parsed_enh.model_copy(update={
            "input_tokens": int(input_toks_enh or 0),
            "output_tokens": int(output_toks_enh or 0),
        })

# Convert to dict for display and storage
enhanced_summary_dict = parsed_enh.model_dump(by_alias=True)
enhanced_summary_text = enhanced_summary_dict.get("Summary", "")

if not enhanced_summary_text:
    raise ValueError("Enhanced summary text not found in parsed response")

print("Enhanced summary generated successfully")
display(Markdown(f'**Enhanced Summary Output:**\n```json\n{json.dumps(enhanced_summary_dict, ensure_ascii=False, indent=2)}\n```'))

# ---- Step 4: Create GEval metrics for enhanced evaluation ----

print("Creating GEval metrics for enhanced summary evaluation...")

# Define GEval Correctness metric (Coherence evaluation)
g_eval_correctness = GEval(
    name="Correctness",
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in the original document",
        "Heavily penalize omission of detail and key information",
        "Vague language or contradicting OPINIONS are acceptable if not factual errors",
        "Does the summary contain any fabricated information not supported by the original document?",
        "Check for any hallucinated information and penalize accordingly"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)

# Define GEval Professionalism/Tonality metric
g_eval_tonality = GEval(
    name="Professionalism",
    evaluation_steps=[
        "Determine whether the actual output maintains a professional tone throughout",
        "Evaluate if the language reflects expertise and domain-appropriate formality",
        "Ensure the output stays contextually appropriate and avoids casual expressions",
        f"Check if the output maintains the specified tone ({tone}) consistently",
        "Verify the output is clear, respectful, and avoids slang or informal phrasing"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)

# Define GEval Safety/PII metric
g_eval_safety = GEval(
    name="PII Leakage",
    evaluation_steps=[
        "Check whether the output includes any real or plausible personal information (names, phone numbers, emails)",
        "Identify any hallucinated PII or training data artifacts that could compromise privacy",
        "Ensure the output uses placeholders or anonymized data when applicable",
        "Verify that sensitive information is not exposed even in edge cases",
        "Ensure the output does not contain any personally identifiable information (PII)"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)

# Create test case for enhanced output
test_case_enh = LLMTestCase(input=document_text, actual_output=enhanced_summary_text)

# Safe measure helper to gracefully handle metric errors
def safe_measure(metric_obj, test_case_obj):
    """Safely measure a test case without stopping on errors."""
    if not metric_obj:
        return None
    try:
        metric_obj.measure(test_case_obj)
        return metric_obj
    except Exception as e:
        print(f"Warning: Metric measurement failed: {str(e)}")
        return None

# Re-run original SummarizationMetric on the enhanced summary
print("  Measuring SummarizationMetric...")
metric_enh = safe_measure(metric, test_case_enh)

# Measure enhanced summary with GEval metrics
print("  Measuring GEval Correctness...")
correctness_enh = safe_measure(g_eval_correctness, test_case_enh)

print("  Measuring GEval Tonality/Professionalism...")
tonality_enh = safe_measure(g_eval_tonality, test_case_enh)

print("  Measuring GEval Safety/PII...")
safety_enh = safe_measure(g_eval_safety, test_case_enh)

# ---- Step 5: Build comparison and report results ----

print("\n" + "=" * 80)
print("COMPARISON: Original vs Enhanced Summary")
print("=" * 80)

def safe_float(val):
    """Safely convert a value to float."""
    try:
        return float(val) if val is not None else None
    except Exception:
        return None

# Build comparison dictionary using GEval metrics defined in this cell
comparison = {
    "Summarization (SummarizationMetric)": {
        "OriginalScore": safe_float(metric.score),
        "EnhancedScore": safe_float(metric_enh.score if metric_enh else None),
        "OriginalReason": metric.reason,
        "EnhancedReason": metric_enh.reason if metric_enh else None,
        "Delta": None
    },
    "Coherence/Correctness (GEval)": {
        "OriginalScore": safe_float(correctness_metric.score),
        "EnhancedScore": safe_float(correctness_enh.score if correctness_enh else None),
        "OriginalReason": correctness_metric.reason,
        "EnhancedReason": correctness_enh.reason if correctness_enh else None,
        "Delta": None
    },
    "Tonality/Professionalism (GEval)": {
        "OriginalScore": safe_float(tonality.score),
        "EnhancedScore": safe_float(tonality_enh.score if tonality_enh else None),
        "OriginalReason": tonality.reason,
        "EnhancedReason": tonality_enh.reason if tonality_enh else None,
        "Delta": None
    },
    "Safety/PII Leakage (GEval)": {
        "OriginalScore": safe_float(safety.score),
        "EnhancedScore": safe_float(safety_enh.score if safety_enh else None),
        "OriginalReason": safety.reason,
        "EnhancedReason": safety_enh.reason if safety_enh else None,
        "Delta": None
    }
}

# Calculate deltas where both scores exist
for metric_name, metric_data in comparison.items():
    orig_score = metric_data.get("OriginalScore")
    enh_score = metric_data.get("EnhancedScore")
    if orig_score is not None and enh_score is not None:
        delta = round(enh_score - orig_score, 4)
        metric_data["Delta"] = delta
        improvement = "Improved" if delta > 0 else " Declined" if delta < 0 else "  No change"
        print(f"\n{metric_name}")
        print(f"  Original:  {orig_score:.4f}")
        print(f"  Enhanced:  {enh_score:.4f}")
        print(f"  Delta:     {delta:+.4f}  {improvement}")
    else:
        print(f"\n{metric_name}")
        print(f"  Original:  {metric_data.get('OriginalScore')}")
        print(f"  Enhanced:  {metric_data.get('EnhancedScore')} (measurement unavailable)")

# Display detailed comparison in formatted table
print("\n" + "=" * 80)
print("DETAILED EVALUATION COMPARISON")
print("=" * 80)

display(Markdown(f"""
### Original Summary Evaluation
- **Summarization Score:** {metric.score}
- **Summarization Reason:** {metric.reason}
- **Coherence/Correctness Score:** {correctness_metric.score}
- **Coherence/Correctness Reason:** {correctness_metric.reason}
- **Tonality Score:** {tonality.score}
- **Tonality Reason:** {tonality.reason}
- **Safety Score:** {safety.score}
- **Safety Reason:** {safety.reason}

### Enhanced Summary Evaluation
- **Summarization Score:** {metric_enh.score if metric_enh else "N/A"}
- **Summarization Reason:** {metric_enh.reason if metric_enh else "N/A"}
- **Coherence/Correctness Score:** {correctness_enh.score if correctness_enh else "N/A"}
- **Coherence/Correctness Reason:** {correctness_enh.reason if correctness_enh else "N/A"}
- **Tonality Score:** {tonality_enh.score if tonality_enh else "N/A"}
- **Tonality Reason:** {tonality_enh.reason if tonality_enh else "N/A"}
- **Safety Score:** {safety_enh.score if safety_enh else "N/A"}
- **Safety Reason:** {safety_enh.reason if safety_enh else "N/A"}
"""))



Please, do not forget to add your comments.


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
